# KI Project
Determining how many cars move in and out of Zurich in relation to weather data


In [2]:
# Libraries
import os
import re
import fnmatch
import datetime
import numpy as np
import pandas as pd
import glob
import holidays
import platform
import socket
from platform import python_version
from datetime import datetime


# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Cars Data
Data Source: https://data.stadt-zuerich.ch/dataset/sid_dav_verkehrszaehlung_miv_od2031

Download files from 2012 to 2021

### Testing Cleaning functions Cars

In [ ]:
'''#Test
fpvk19 = 'Data\\cars_Zurich\\vk_2019.csv'
df = pd.read_csv(fpvk19, sep=',', encoding='utf-8')
df.shape'''

In [ ]:
'''# Count missing values
print('Count missing values per variable')
print(pd.isna(df).sum(), '\n')

# Identify rows with missing values
print('Identify rows with missing values')
print(df.loc[df.isna().any(axis=1)][['AnzFahrzeuge']], '\n')

# Drop rows where at least one element is missing.
df2 = df.dropna()
df2.head()'''

In [ ]:
# Count missing values
#print('Count missing values per variable')
#print(pd.isna(df2).sum(), '\n')

In [ ]:
#cols_to_remove = ['MSID', 'MSName', 'ZSID','Achse','HNr','Hoehe', 'EKoord', 'NKoord', 'AnzDetektoren', 'D1ID', 'D2ID','D3ID','D4ID','LieferDat','Knummer']
#df = df.drop(columns=cols_to_remove)

In [ ]:
#df.head()

In [ ]:
#unique_values = np.unique(df['AnzFahrzeugeStatus'])
#print(unique_values)

In [ ]:
#column_name1 = 'Richtung'
#specific_values1= ['auswärts', 'einwärts']
#column_name2 = 'AnzFahrzeugeStatus'
#specific_values2= ['Gemessen', 'Imputiert']

# Filter the DataFrame to show only rows with the specific values in the specified column
#df = df[df[column_name1].isin(specific_values1) & df[column_name2].isin(specific_values2)]

#df = df[df['AnzFahrzeuge']!=0] 

#df.shape

In [ ]:
#df.to_csv('test.csv', index=False)

In [ ]:
#filtered_df.head()

#filtered_df.to_csv('filtered_cars.csv', index=False)

In [ ]:
# Group the DataFrame by 'Richtung' and 'MessungDatZeit' columns and calculate the sum of 'AnzFahrzeuge'
#sum_df = df.groupby(['Richtung', 'MessungDatZeit'])['AnzFahrzeuge'].sum().reset_index()

#sum_df.shape

#sum_df.to_csv('sum.csv', index=False)

In [ ]:
#summary_df.head()

### Create cleaning function for cars data

In [ ]:
# define a cleaning function

def clean_data_vk(df):
    cols_to_remove = ['MSID', 'MSName', 'ZSID','Achse','HNr','Hoehe', 'EKoord', 'NKoord', 'AnzDetektoren', 'D1ID', 'D2ID','D3ID','D4ID','LieferDat','Knummer']
    df = df.drop(columns=cols_to_remove)

    # Only use Richtungen 'auswärts' and 'einwärts'
    column_name1 = 'Richtung'
    specific_values1= ['auswärts', 'einwärts']

    # Only Use Status 'Gemessen' and 'Imputiert'
    column_name2 = 'AnzFahrzeugeStatus'
    specific_values2= ['Gemessen', 'Imputiert']

    #apply filter 
    df = df[df[column_name1].isin(specific_values1) & df[column_name2].isin(specific_values2)]

    # COMMENT ON THIS -----------------------------------------  
    # remove rows where no data has been collected
    # df = df[df['AnzFahrzeuge']!=0] 

    # Group the DataFrame by 'Richtung' and 'MessungDatZeit' columns and calculate the sum of 'AnzFahrzeuge'
    df = df.groupby(['Richtung', 'MessungDatZeit'])['AnzFahrzeuge'].sum().reset_index()

    df.rename(columns={'MessungDatZeit': 'DateTime'}, inplace=True)

    # Change the Value of 'Richtung' to Binary 1 = einwärts, 0 = auswärts
    df['Richtung'] = df['Richtung'].map(lambda x: 0 if x == 'auswärts' else 1)

    return df
    

### Execute cleaning on all car data files 

In [ ]:
# Specify the folder path where CSV files are located
fpvk = 'Data\\cars_Zurich\\'

#Choosing the target directory for cleaned files 
target_directory = 'Data\\cars_Zurich\\cleaned\\'

# Create the target directory if it does not exist
os.makedirs(target_directory, exist_ok=True)

# Get a list of all CSV files in the folder
csv_files = glob.glob(fpvk + '*.csv')

# Iterate over each CSV file
for file in csv_files:

    # Read the CSV file into a DataFrame
    df = pd.read_csv(file)
    
    # Apply data cleaning function
    cleaned_df = clean_data_vk(df)
    
    # Save cleaned data to a new CSV file
    file_name = os.path.basename(file)  # Get the file name
    new_file_path = os.path.join(target_directory, file_name.replace('.csv', '_cleaned.csv'))
    cleaned_df.to_csv(new_file_path, index=False)

### Merge cleaned cars data

In [ ]:
# Define file path
fpvk = 'Data\\cars_Zurich\\cleaned\\'

# Get a list of all CSV files in the folder
csv_files = glob.glob(fpvk + '*.csv')

# Create an empty list to store the individual DataFrames
dataframes = []

# Iterate over each CSV file
for file in csv_files:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file)
    
    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged DataFrame as a CSV file
merged_df.to_csv('merged_cars_data.csv', index=False)

In [ ]:
merged_df.shape

In [ ]:
merged_df.head()

In [ ]:
print(merged_df['DateTime'].dtype)
print(merged_df['Richtung'].dtype)
print(merged_df['AnzFahrzeuge'].dtype)


# Weather data

Data Source: https://data.stadt-zuerich.ch/dataset/ugz_meteodaten_stundenmittelwerte

Download files from 2012 to 2021

### Cleaning Function for weather data

In [ ]:
# define a cleaning function
def clean_data_w(df):

    #remove unused cols
    cols_to_remove_w = ['Intervall','Parameter']
    df = df.drop(columns=cols_to_remove_w)

    # Merge rows into cols
    # Create a new column as a combination of 'col1' and 'col2' values
    df['col1_col2_col5'] = df['Datum'] + '_' + df['Standort'] + '_' + df['Status']

    # Use pivot_table to merge multiple rows into one row and create new columns
    df_w = pd.pivot_table(df, index=['Datum', 'Standort'], columns='Einheit', values='Wert', aggfunc=lambda x: x.iloc[0]).reset_index()

    #reset ID column to none
    df_w.columns.name = None

    #remove unused columns
    cols_to_remove_w = ['W/m2','m/s','°']
    df_w = df_w.drop(columns=cols_to_remove_w)

    #rename Columns
    df_w.rename(columns={'min': 'Niederschlag'}, inplace=True)
    df_w.rename(columns={'%Hr': 'Luftfeuchtigkeit (%Hr)'}, inplace=True)
    df_w.rename(columns={'hPa': 'Luftdruck (hPa)'}, inplace=True)
    df_w.rename(columns={'°C': 'Temperatur (°C)'}, inplace=True)
    df_w.rename(columns={'Datum': 'DateTime'}, inplace=True)

    #average out the values 
    df_w = df_w.groupby('DateTime').mean().round(2)

    #change niederschlag from float to binary 1 = Rain, 0 = No rain
    df_w['Niederschlag'] = df_w['Niederschlag'].map(lambda x: 0 if x == 0 else 1)

    df_w = df_w.reset_index()

    return df_w

### Executing Cleaning for all weather data files

In [ ]:
# Specify the folder path where CSV files are located
fpw = 'Data\\weather_Zurich\\'

#Choosing the target directory for cleaned files 
target_directory = 'Data\\weather_Zurich\\cleaned\\'

# Create the target directory if it does not exist
os.makedirs(target_directory, exist_ok=True)

# Get a list of all CSV files in the folder
csv_files = glob.glob(fpw + '*.csv')

# Iterate over each CSV file
for file in csv_files:

    # Read the CSV file into a DataFrame
    df_w = pd.read_csv(file)
    
    # Apply data cleaning function
    cleaned_df_w = clean_data_w(df_w)
    
    # Save cleaned data to a new CSV file
    file_name = os.path.basename(file)  # Get the file name
    new_file_path = os.path.join(target_directory, file_name.replace('.csv', '_cleaned.csv'))
    cleaned_df_w.to_csv(new_file_path, index=False)

### Merging all cleaned weather data files

In [ ]:
#Specify folder of cleaned data
cfpw = 'Data\\weather_Zurich\\cleaned\\'

# Get a list of all CSV files in the folder
csv_files_c = glob.glob(cfpw + '*.csv')

# Create an empty list to store the individual DataFrames
dataframes = []

# Iterate over each CSV file
for file in csv_files_c:
    # Read the CSV file into a DataFrame
    df_w = pd.read_csv(file)
    
    # Append the DataFrame to the list
    dataframes.append(df_w)

# Concatenate all DataFrames into a single DataFrame
merged_df_w = pd.concat(dataframes, ignore_index=True, sort=False)

# Save the merged DataFrame as a CSV file
merged_df_w.to_csv('merged_weather_data.csv', index=False)

In [ ]:
merged_df_w.shape

In [ ]:
merged_df_w.head()

In [ ]:
print(merged_df_w['DateTime'].dtype)
print(merged_df_w['Luftfeuchtigkeit (%Hr)'].dtype)
print(merged_df_w['Luftdruck (hPa)'].dtype)
print(merged_df_w['Niederschlag'].dtype)
print(merged_df_w['Temperatur (°C)'].dtype)


###  Testing Weather Data

In [ ]:
"""#Test
fpw21 = 'Data\\weather_Zurich\\w_2021.csv'
df = pd.read_csv(fpw21, sep=',', encoding='utf-8')
df.shape"""

In [ ]:
""""#remove unused cols
cols_to_remove_w = ['Intervall','Parameter']
df = df.drop(columns=cols_to_remove_w)

# Merge rows into cols
# Create a new column as a combination of 'col1' and 'col2' values
df['col1_col2_col5'] = df['Datum'] + '_' + df['Standort'] + '_' + df['Status']

# Use pivot_table to merge multiple rows into one row and create new columns
df = pd.pivot_table(df, index=['Datum', 'Standort'], columns='Einheit', values='Wert', aggfunc=lambda x: x.iloc[0]).reset_index()

#reset ID column to none
df.columns.name = None

#remove unused columns
cols_to_remove_w = ['W/m2','m/s','°','Standort']
df = df.drop(columns=cols_to_remove_w)

#rename Columns
df.rename(columns={'min': 'Niederschlag'}, inplace=True)
df.rename(columns={'%Hr': 'Luftfeuchtigkeit (%Hr)'}, inplace=True)
df.rename(columns={'hPa': 'Luftdruck (hPa)'}, inplace=True)
df.rename(columns={'°C': 'Temperatur (°C)'}, inplace=True)
df.rename(columns={'Datum': 'DateTime'}, inplace=True)

#average out the values 
df = df.groupby('DateTime')['Niederschlag', 'Luftfeuchtigkeit (%Hr)', 'Luftdruck (hPa)', 'Temperatur (°C)'].mean().round(2)

#change niederschlag from float to boolean
df['Niederschlag'] = df['Niederschlag'].map(lambda x: False if x == 0 else True)
"""



In [ ]:
"""df = df.reset_index()
df.head()"""

# Merging two datasets and Feature Engineering

In [ ]:
# Read the first file into a DataFrame
df1 = pd.read_csv('merged_cars_data.csv')

# Read the second file into a DataFrame
df2 = pd.read_csv('merged_weather_data.csv')

# Convert the date and time columns in df1 to a consistent format
df1['DateTime'] = pd.to_datetime(df1['DateTime'], format='%Y-%m-%dT%H:%M:%S')

# Convert the date and time columns in df2 to a consistent format
df2['DateTime'] = pd.to_datetime(df2['DateTime'], format='%Y-%m-%dT%H:%M%z')


In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
#Change dtype of weather data DateTime
df2['DateTime'] = df2['DateTime'].dt.tz_localize(None)

#douple check correct dtype 
print(df2['DateTime'].dtype)
print(df1['DateTime'].dtype)

In [ ]:
# Merge the two DataFrames based on the common columns
merged_df = pd.merge(df1, df2, on='DateTime')

merged_df.head()

In [ ]:
# Create a new column 'Time' containing the time component
merged_df['Time'] = merged_df['DateTime'].dt.time
merged_df['Time'] = pd.to_datetime(merged_df['Time'], format='%H:%M:%S').dt.time

# Create new column 'Date' containing the Date component
merged_df['Date'] = merged_df['DateTime'].dt.date
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

# Drop the Column DateTime
merged_df = merged_df.drop(columns='DateTime')

In [ ]:
# Create a new Column 'holiday' to check if the day was a public swiss holiday
swiss_holidays = holidays.CountryHoliday('CH')
merged_df['holiday'] = merged_df['Date'].apply(lambda x: 1 if x.date() in swiss_holidays else 0)

merged_df.head()

In [ ]:
print(merged_df.dtypes)

In [ ]:
# Count missing values
print('Count missing values per variable')
print(pd.isna(merged_df).sum(), '\n')

# Identify rows with missing values
print('Identify rows with missing values')
print(merged_df.loc[merged_df.isna().any(axis=1)][['Richtung', 'Date', 'Time', 'AnzFahrzeuge', 'Luftfeuchtigkeit (%Hr)', 'Luftdruck (hPa)', 'Niederschlag', 'Temperatur (°C)']], '\n')


In [ ]:
#Remove the rows with missing values 
merged_df = merged_df.dropna()

In [ ]:
# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_data.csv', index=False)

# Feature Creation

In [3]:
df = pd.read_csv('merged_data.csv')

df.shape

(174914, 11)

In [4]:
df.head()

,Richtung,AnzFahrzeuge,Luftfeuchtigkeit (%Hr),Luftdruck (hPa),Niederschlag,Temperatur (°C),Time,Date,holiday,Hour,Weekday
0,0,4043.0,92.51,970.12,1,7.58,00:00:00,2012-01-01,1,0,6
1,1,2931.0,92.51,970.12,1,7.58,00:00:00,2012-01-01,1,0,6
2,0,11021.0,91.00,970.09,1,7.98,01:00:00,2012-01-01,1,1,6
3,1,6660.0,91.00,970.09,1,7.98,01:00:00,2012-01-01,1,1,6
4,0,7892.0,90.38,970.46,0,7.62,02:00:00,2012-01-01,1,2,6


In [5]:
# Creating a new column 'Hour' representing the hour as an integer
df['Hour'] = df['Time'].apply(lambda x: int(x.split(':')[0]))

# Creating a new column for weekday 0-6
df['Weekday'] = pd.to_datetime(df['Date']).dt.weekday

# Creating new column for year 
df['Year'] = df['Date'].apply(lambda x: int(x.split('-')[0]))

# Creating new column for month
df['Month'] = df['Date'].apply(lambda x: int(x.split('-')[1]))

# Creating new column for day
df['Day'] = df['Date'].apply(lambda x: int(x.split('-')[2]))

In [8]:
df.head()

,Richtung,AnzFahrzeuge,Luftfeuchtigkeit (%Hr),Luftdruck (hPa),Niederschlag,Temperatur (°C),Time,Date,holiday,Hour,Weekday,Year,Month,Day
0,0,4043.0,92.51,970.12,1,7.58,00:00:00,2012-01-01,1,0,6,2012,1,1
1,1,2931.0,92.51,970.12,1,7.58,00:00:00,2012-01-01,1,0,6,2012,1,1
2,0,11021.0,91.00,970.09,1,7.98,01:00:00,2012-01-01,1,1,6,2012,1,1
3,1,6660.0,91.00,970.09,1,7.98,01:00:00,2012-01-01,1,1,6,2012,1,1
4,0,7892.0,90.38,970.46,0,7.62,02:00:00,2012-01-01,1,2,6,2012,1,1


In [6]:
# overwrite the existing merged_data.csv
df.to_csv('merged_data.csv', index=False)

In [9]:
df.dtypes

Richtung                    int64
AnzFahrzeuge              float64
Luftfeuchtigkeit (%Hr)    float64
Luftdruck (hPa)           float64
Niederschlag                int64
Temperatur (°C)           float64
Time                       object
Date                       object
holiday                     int64
Hour                        int64
Weekday                     int64
Year                        int64
Month                       int64
Day                         int64
dtype: object

In [7]:
print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
NT
Windows | 10
Datetime: 2023-06-17 02:33:56
Python Version: 3.10.11
-----------------------------------
